In [1]:
from typing import Callable, Optional, List
import copy
import math

import numpy as np
import torch
import torch.nn.functional as F
import jieba
from tqdm import tqdm

from configuration_enc_dec import EncDecConfig
from tokenization_enc_dec import EncDecTokenizer
from model import TorchEncDecModel, top_k_logits

In [2]:
# !pip install jieba --user
jieba.initialize()

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.564 seconds.
Prefix dict has been built successfully.


In [3]:
config = EncDecConfig(
    d_model=4096,
    d_ff=10240,
    d_kv=64,
    num_heads=64,
    num_layers=24,
    num_decoder_layers=24,
    dropout_rate=0.0,
    feed_forward_proj="relu",
    init_method_std=0.001,
    initializer_factor=1.0,
    layer_norm_epsilon=1e-06,
    max_position_embeddings=512,
    use_cache=True,
    use_scaled_init_for_output_weights=True,
    do_dim_trick=False
)
config.vocab_size = 26240
config.vocab_size 

26240

In [4]:
print('build model')
model = TorchEncDecModel(config)

build model


In [5]:
# To float16 and GPU
if torch.cuda.is_available():
    model = model.half().cuda()

In [6]:
print('load state')
model.load_state_dict(torch.load('converted.zip'))
# model.load_state_dict(state_dict_new)

load state


<All keys matched successfully>

In [7]:
print('to eval')
model = model.eval()

to eval


In [8]:
tokenizer = EncDecTokenizer('./vocab.txt')


def get_enc_hidden_state(input_prompt, dtype):
    inputs = tokenizer.encode(input_prompt)
    enc_input_ids = torch.tensor([
        inputs
    ], dtype=torch.long)
    inputs_len = enc_input_ids.shape[1]
    enc_attention_mask = torch.ones(1, 1, inputs_len, inputs_len, dtype=dtype)
    if torch.cuda.is_available():
        enc_input_ids = enc_input_ids.cuda()
        enc_attention_mask = enc_attention_mask.cuda()
    enc_outputs = model(enc_input_ids, only_encoder=True, enc_attention_mask=enc_attention_mask)
    del enc_input_ids
    del enc_attention_mask
    enc_hidden_states = enc_outputs['encoder_last_hidden_state']
    del enc_outputs
    return inputs_len, enc_hidden_states


def predict(input_prompt, output_prompt, length=100):
    
    dtype = torch.float32
    if torch.cuda.is_available():
        dtype = torch.float16

    inputs_len, enc_hidden_states = get_enc_hidden_state(input_prompt, dtype)
    outputs = [1, tokenizer.get_sentinel_id(0)] + tokenizer.encode(output_prompt)
    past_key_values = None
    raw_outputs_len = len(outputs)
    out_texts = ''

    for i in tqdm(range(length - raw_outputs_len)):

        dec_input_ids = torch.tensor([
            outputs
        ], dtype=torch.long)

        outputs_len = i + raw_outputs_len
        if i == 0:
            outputs_len = len(outputs)

        cross_attention_mask = torch.zeros(1, 1, outputs_len, inputs_len, dtype=dtype)
        cross_attention_mask[0, 0, :outputs_len, :inputs_len] = 1.0

        dec_attention_mask = torch.zeros(1, 1, outputs_len, outputs_len, dtype=dtype)
        dec_attention_mask[0][0] = torch.tril(torch.ones(outputs_len, outputs_len))

        if i > 0:
            cross_attention_mask = cross_attention_mask[:, :, -1:, :]
            dec_attention_mask = dec_attention_mask[:, :, -1:, :]

        if torch.cuda.is_available():
            dec_input_ids = dec_input_ids.cuda()
            dec_attention_mask = dec_attention_mask.cuda()
            cross_attention_mask = cross_attention_mask.cuda()

        out = model(
            dec_input_ids=dec_input_ids,
            dec_attention_mask=dec_attention_mask,
            cross_attention_mask=cross_attention_mask,
            enc_hidden_states=enc_hidden_states,
            past_key_values=past_key_values
        )
        
        del dec_input_ids
        del dec_attention_mask
        del cross_attention_mask
        del past_key_values
        past_key_values = out['past_key_values']

        temperature = 1.0
        logits = out['lm_logits'][:, -1, :].detach().cpu().to(torch.float32)
        logits[:, 0] = -10000
        logits[:, 26050:] = -10000
        del out

        next_token_logscores = top_k_logits(logits / temperature, top_k=10, top_p=0.9)
        probs = F.softmax(next_token_logscores, dim=-1)
        next_token = torch.multinomial(probs.float(), num_samples=1).squeeze(1)
        next_token_list = next_token.detach().cpu().numpy().tolist()

        outputs = next_token_list
        out_text = tokenizer.decode(outputs)
        out_texts += out_text

    del enc_hidden_states
    return out_texts

In [9]:
input_prompt = '''外国人犯罪案件归基层检察机关管辖吗？
根据中华人民共和国法律，基层检察机关可以办理外国人犯罪案件，涉外因素不是影响管辖的法定事由。
《<中华人民共和国刑事诉讼法>解释》第393条：第一审涉外刑事案件，除刑事诉讼法第二十条至第二十二条规定的以外，由基层人民法院管辖。必要时，中级人民法院可以指定辖区内若干基层人民法院集中管辖第一审涉外刑事案件，也可以依照刑事诉讼法第二十三条的规定，审理基层人民法院管辖的第一审涉外刑事案件。
人民检察院管辖范围与同级人民法院一致。
外国人犯罪在犯罪构成、罪名和刑期方面与我国公民犯罪有什么区别吗？
没有区别。根据中华人民共和国法律，对任何人犯罪，在适用法律上一律平等。外国籍身份不是影响事实认定、犯罪构成和判决刑期的法定事由。
《中华人民共和国刑法》第4条：对任何人犯罪，在适用法律上一律平等。不允许任何人有超越法律的特权。
《中华人民共和国刑事诉讼法》第6条：人民法院、人民检察院和公安机关进行刑事诉讼，必须依靠群众，必须以事实为根据，以法律为准绳。对于一切公民，在适用法律上一律平等，在法律面前，不允许有任何特权。'''
output_prompt = '外国人在中国不能做什么呢？'

In [10]:
out_texts = predict(input_prompt, output_prompt, 50)
if torch.cuda.is_available():
    torch.cuda.empty_cache()

/home/jovyan/work/zy/new_convert/model.py:280: UserWarning: Mixed memory format inputs detected while calling the operator. The operator will output contiguous tensor even if some of the inputs are in channels_last format. (Triggered internally at  /pytorch/aten/src/ATen/native/TensorIterator.cpp:918.)
  attention_scores = torch.mul(
100%|██████████| 39/39 [00:32<00:00,  1.20it/s]


In [11]:
print(input_prompt)

外国人犯罪案件归基层检察机关管辖吗？
根据中华人民共和国法律，基层检察机关可以办理外国人犯罪案件，涉外因素不是影响管辖的法定事由。
《<中华人民共和国刑事诉讼法>解释》第393条：第一审涉外刑事案件，除刑事诉讼法第二十条至第二十二条规定的以外，由基层人民法院管辖。必要时，中级人民法院可以指定辖区内若干基层人民法院集中管辖第一审涉外刑事案件，也可以依照刑事诉讼法第二十三条的规定，审理基层人民法院管辖的第一审涉外刑事案件。
人民检察院管辖范围与同级人民法院一致。
外国人犯罪在犯罪构成、罪名和刑期方面与我国公民犯罪有什么区别吗？
没有区别。根据中华人民共和国法律，对任何人犯罪，在适用法律上一律平等。外国籍身份不是影响事实认定、犯罪构成和判决刑期的法定事由。
《中华人民共和国刑法》第4条：对任何人犯罪，在适用法律上一律平等。不允许任何人有超越法律的特权。
《中华人民共和国刑事诉讼法》第6条：人民法院、人民检察院和公安机关进行刑事诉讼，必须依靠群众，必须以事实为根据，以法律为准绳。对于一切公民，在适用法律上一律平等，在法律面前，不允许有任何特权。


In [12]:
print(output_prompt + out_texts)

外国人在中国不能做什么呢？
在法律面前，每个公民都是平等的，任何人都不能作超越法律的特权。
人民检察院可以办理外国人犯罪案件。但不能对外国人
